### V Function

In [ ]:
def v_function(A, C_values):
    """
    Computes worth of one coalition of channels
    :param A: a coalition of channels
    :param C_values: dict with conversion # of each set of channels
    :return: worth of A
    """
    def subseq(seq):
        n = len(seq)
        for i in range(1, 2**n):
            b = format(i, f'0{n}b')
            s = []
            for j in range(len(b)):
                if int(b[-j-1]) == 1:
                    s.append(seq[j])
            yield s
    worth_of_A = 0
    for subset in subseq(A.split(',')):
        subset.sort()
        subset = ','.join(subset)
        if subset in C_values.keys():
            worth_of_A += C_values[subset]
    return worth_of_A

### Import dataset

In [ ]:
import pandas as pd

df = pd.read_csv('shapley_source.csv')
C_values = df.set_index('copy_list').to_dict()['conversions']
channels = [c for c in C_values.keys() if ',' not in c]
print(sum(C_values.values()))

In [ ]:
def get_v_values(C_values):
    from collections import defaultdict
    import os.path
    import pickle
    filename = 'v_values.pickle'
    if os.path.isfile(filename):
        with open(filename, 'rb') as file:
            return pickle.load(file)
    v_values = defaultdict(float)
    count = 0
    big = []
    for A in C_values.keys():
        if len(A.split(',')) > 21:
            big.append({A: C_values[A]})
            continue
        count += 1
        print(f'{count}: {A}\r', end='')
        v_values[A] = v_function(A, C_values)
    print('')
    print(v_values)
    print(f'big = {big}')
    with open(filename, 'wb') as file:
        pickle.dump(v_values, file)
    return v_values
    
v_values = get_v_values(C_values)
print(v_values)

In [ ]:
def shapley(channels, v_values):
    from collections import defaultdict
    from math import factorial
    n = len(channels)
    res = defaultdict(int)
    for channel in channels:
        for A in v_values.keys():
            if channel not in A.split(','):
                cardinal_A = len(A.split(','))
                A_with_channel = A.split(',')
                A_with_channel.append(channel)
                A_with_channel = ','.join(sorted(A_with_channel))
                res[channel] += (v_values[A_with_channel] - v_values[A])*(factorial(n-cardinal_A-1)/factorial(n))
    return res

In [ ]:
s = shapley(channels, v_values)
print(s)
print(sum(s.values()))

### Plot Result Histogram

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(30, 10))
ax = fig.add_subplot(111)
ax.bar(s.keys(), s.values(), color='r')
plt.xlabel('Copy')
plt.ylabel('Conversions')
plt.savefig('conversions.png', format='png')
plt.show()